In [127]:
import pandas as pd
import numpy as np

In [128]:
import os
cwd = os.getcwd()
print(cwd)

/Users/rosamund/Documents/GitHub/uk-electricity-map


In [129]:
df1 = pd.read_csv((cwd + "/uk-map.csv"))
df1.head(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
0,Aberdare District Energy,EDF Energy,Gas,NaN,10.0,51.697527,-3.416181,No,NaN,2002.0,2005,2014
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1455.0,51.387006,-3.406215,No,NaN,1971.0,2004,2007
2,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008,2017


In [130]:
df1.tail(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
6799,Westhampnett Solar Farm Battery Storage,West Sussex County Council (100%),Storage,Battery,4.4,50.8469,-0.745574,Yes,No,2018.0,2018,2019
6800,Willenhall Battery Storage Demonstration,University of Sheffield (100%),Storage,Battery,1.0,52.5778,-2.049130,Yes,No,2016.0,2016,2019
6801,Willersey Energy Storage Facility,Belectric (100%),Storage,Battery,1.4,52.0655,-1.849880,Yes,No,2017.0,2017,2019


In [131]:
#think appearance of excess rows may due to Simon's treament of the data carrying across
#update: it's an issue with how Simon exported the CSV at row 1770

df1.shape

(6802, 12)

In [132]:
df1.tail(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
6799,Westhampnett Solar Farm Battery Storage,West Sussex County Council (100%),Storage,Battery,4.4,50.8469,-0.745574,Yes,No,2018.0,2018,2019
6800,Willenhall Battery Storage Demonstration,University of Sheffield (100%),Storage,Battery,1.0,52.5778,-2.049130,Yes,No,2016.0,2016,2019
6801,Willersey Energy Storage Facility,Belectric (100%),Storage,Battery,1.4,52.0655,-1.849880,Yes,No,2017.0,2017,2019


In [133]:
df2 = df1.rename(index=str, columns={"Site": "site", "Company": "operator", "Fuel": "type", "Type": "fuelDetail", "Capacity (MW)": "capacity", "Latitude": "latitude", "Longitude": "longitude", "Low-Carbon": "lowCarbon", "CHP": "chp", "Year Open": "yearOpen", "yearStart": "yearStart", "yearEnd": "yearEnd"})
df2.head(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
0,Aberdare District Energy,EDF Energy,Gas,NaN,10.0,51.697527,-3.416181,No,NaN,2002.0,2005,2014
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1455.0,51.387006,-3.406215,No,NaN,1971.0,2004,2007
2,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008,2017


In [134]:
df2.tail(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
6799,Westhampnett Solar Farm Battery Storage,West Sussex County Council (100%),Storage,Battery,4.4,50.8469,-0.745574,Yes,No,2018.0,2018,2019
6800,Willenhall Battery Storage Demonstration,University of Sheffield (100%),Storage,Battery,1.0,52.5778,-2.049130,Yes,No,2016.0,2016,2019
6801,Willersey Energy Storage Facility,Belectric (100%),Storage,Battery,1.4,52.0655,-1.849880,Yes,No,2017.0,2017,2019


In [135]:
#find out if any nulls for long/lat
sum(df2['longitude'].isna())

69

In [136]:
#drop rows where longitude is blank

df2 = df2.dropna(subset=['longitude'])
sum(df2['longitude'].isna())

0

In [137]:
#check latitude doesn't have NaN either

sum(df2['latitude'].isna())

0

In [138]:
#shape as expected
df2.shape

(6733, 12)

In [119]:
#replace gaps with hyphens

df2 = df2.fillna("-")

In [120]:
 df2.to_csv('uk-map-edit.csv', index=False)

In [121]:
#begin process of converting to geojson

import geopandas as gpd

In [122]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [123]:
geometry = [Point(xy) for xy in zip(df2.longitude, df2.latitude)]
df2 = df2.drop(['longitude', 'latitude'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df2, crs=crs, geometry=geometry)
gdf.head(3)

,site,operator,type,fuelDetail,capacity,lowCarbon,chp,yearOpen,yearStart,yearEnd,geometry
0,Aberdare District Energy,EDF Energy,Gas,-,10.0,No,-,2002,2005,2014,POINT (-3.416181 51.697527)
1,Aberthaw B,RWE Npower Plc,Coal,-,1455.0,No,-,1971,2004,2007,POINT (-3.406215 51.387006)
2,Aberthaw B,RWE Npower Plc,Coal,-,1586.0,No,-,1971,2008,2017,POINT (-3.406215 51.387006)


In [124]:
gdf.to_file("data/power_stations.json", driver="GeoJSON")